# Neural Checker User Guide

The aim of this notebook is to provide a step-by-step guide to using the Neural Checker tool. It is developed by the team of Artificial Intelligence and Big Data of the Universidad ORT. Its main goal is to provide implementations for the structures needed for working in the Model Extraction Framework and enable the explainability and checking of complex systems in a black box context.

## Pythautomata

### Installation

In [14]:
%pip install pythautomata

Note: you may need to restart the kernel to use updated packages.


### Automatons

#### Moore machines

##### Dependencies

In [15]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.moore_state import MooreState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy as ComparisonStrategy
from pythautomata.model_exporters.dot_exporters.moore_dot_exporting_strategy import MooreDotExportingStrategy as DotExportingMMStrategy

##### Implementation

In [16]:
# First we create an input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Then the output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

# Defining moore states
stateA = MooreState(name="State A", value=output_alphabet['0'])
stateB = MooreState("State B", output_alphabet['1'])
stateC = MooreState("State C", output_alphabet['1'])

# Adding transitions for the previous states
stateA.add_transition(a, stateA)
stateA.add_transition(b, stateB)
stateB.add_transition(a, stateC)
stateB.add_transition(c, stateB)

# Finaly we create the moore machine with the alphabets and states previously created 
moore_machine = MooreMachineAutomaton(input_alphabet, output_alphabet, initial_state=stateA, 
                                      states=set([stateA, stateB, stateC]), comparator=ComparisonStrategy, 
                                      name="moore machine with 3 states")

# Moore machine is exported in the directory ./output_models/moore_machines/ with .dot extension
moore_machine.export('./output_models/moore_machines/')

#### Mealy machines

##### Dependencies

In [17]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.mealy_state import MealyState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.mealy_machine import MealyMachine
from pythautomata.model_comparators.mealy_machine_comparison_strategy import MealyMachineComparisonStrategy as MealyComparisonStrategy

##### Implementation

In [18]:
# First we create an input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Then the output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

zero = output_alphabet['0']
one = output_alphabet['1']

# Defining mealy states
stateA = MealyState(name="State A")
stateB = MealyState(name="State B")
stateC = MealyState(name="State C")

# Adding transitions for the previous states
stateA.add_transition(a, stateA, zero)
stateA.add_transition(b, stateB, one)
stateB.add_transition(a, stateC, zero)
stateB.add_transition(c, stateB, one)

# Finaly we create the mealy machine with the alphabets and states previously created 
mealy_machine = MealyMachine(input_alphabet, output_alphabet, initial_state=stateA, states=set([stateA, stateB, stateC]), 
                             comparator=MealyComparisonStrategy, 
                             name="mealy machine with 3 states")

# Mealy machine is exported in the directory ./output_models/mealy_machines/ with .dot extension
mealy_machine.export('./output_models/mealy_machines/')

### Generators

#### Moore machine generator

##### Dependencies

In [19]:
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine

##### Implementation

In [20]:
# Generate a random Moore Machine with 500 states
input_alphabet = Alphabet.from_strings(["a", "b", "c"])
output_alphabet = Alphabet.from_strings(['0', '1'])
seed = 100
number_of_states = 500
automata = generate_moore_machine(input_alphabet, output_alphabet, number_of_states, seed)

### Converters and Comparators

#### DFA - Moore machine converter and comparator

##### Dependencies

In [21]:
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy

##### Implementation

In [22]:
# Tomitas DFA example
tomitas = TomitasGrammars.get_automaton_1()

# Convert tomitas DFA to moore machine
converted_moore_machine = AutomataConverter.convert_dfa_to_moore_machine(tomitas)

# Manually created tomitas moore machine
tomitas_moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Comparate tomitas automatons
comparison_strategy = MooreMachineComparisonStrategy()
comparison_strategy.are_equivalent(automaton1=converted_moore_machine, automaton2=tomitas_moore_machine)

True

#### Moore machine - Mealy machine converter and comparator

Dependencies

In [23]:
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines
from pythautomata.model_comparators.mealy_machine_comparison_strategy import MealyMachineComparisonStrategy

Implementation

In [24]:
# Tomitas DFA example
moore_machine = SampleMooreMachines.get_3_states_automaton()

# Convert tomitas DFA to moore machine
converted_mealy_machine = AutomataConverter.convert_moore_machine_to_mealy_machine(moore_machine)

# Manually created mealy machine
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'))))
output_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))

a = input_alphabet['a']
b = input_alphabet['b']

output_a = output_alphabet['a']
output_b = output_alphabet['b']
output_c = output_alphabet['c']

# Defining mealy states
stateA = MealyState(name="State A")
stateB = MealyState(name="State B")
stateC = MealyState(name="State C")

# Adding transitions for the previous states
stateA.add_transition(a, stateB, output_b)
stateA.add_transition(b, stateC, output_c)
stateB.add_transition(a, stateC, output_c)
stateB.add_transition(b, stateA, output_a)
stateC.add_transition(a, stateA, output_a)
stateC.add_transition(b, stateB, output_b)

# Finaly we create the mealy machine with the alphabets and states previously created 
mealy_machine = MealyMachine(input_alphabet, output_alphabet, initial_state=stateA, states=set([stateA, stateB, stateC]), 
                             comparator=MealyComparisonStrategy, 
                             name="mealy machine with 3 states")

# Comparate automatons
comparison_strategy = MealyMachineComparisonStrategy()
comparison_strategy.are_equivalent(machine1=converted_mealy_machine, machine2=mealy_machine)

True

## PyModelExtractor

### Installation

In [25]:
%pip install pymodelextractor

Note: you may need to restart the kernel to use updated packages.


## Observation Table algorithms

### General L*

#### Dependencies

In [26]:
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy
from pythautomata.model_comparators.dfa_comparison_strategy import DFAComparisonStrategy
from pymodelextractor.teachers.general_teacher import GeneralTeacher
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pymodelextractor.factories.lstar_factory import LStarFactory
from pythautomata.utilities.nicaud_dfa_generator import generate_dfa
from pythautomata.base_types.alphabet import Alphabet

#### Implementation using a Moore machine

In [27]:
# Create the General L* learner that implements the L* algorithm
learner = LStarFactory.get_moore_machine_lstar_learner()

# Import a sample Moore Machine
moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Create the teacher for the Moore Machine Learner
mm_teacher = GeneralTeacher(moore_machine, MooreMachineComparisonStrategy())

# Learn the generated Moore Machine
teacher = mm_teacher
result = learner.learn(teacher) 

# Show that the learned Moore Machine is equivalent to the generated Moore Machine
print("Are equal: ", MooreMachineComparisonStrategy().are_equivalent(
    result.model, moore_machine))

# Export the learned Moore Machine
result.model.export('./output_models/mm_lstar/')

Are equal:  True


#### Implementation using a DFA

In [28]:
# Create the General L* learner that implements the L* algorithm
learner = LStarFactory.get_dfa_lstar_learner()

# Import a sample DFA
dfa =  TomitasGrammars.get_automaton_1()

# Create the teacher for the DFA learner
dfa_teacher = GeneralTeacher(dfa, DFAComparisonStrategy())

# Learn the generated DFA
teacher = dfa_teacher
result = learner.learn(teacher) 

# Show that the learned DFA is equivalent to the generated DFA
print("Are equal: ", DFAComparisonStrategy().are_equivalent(
    result.model, dfa))

# Export the learned DFA
result.model.export('./output_models/dfa_lstar/')

Are equal:  True


#### Partial L*

The partial L* algorithm enables the inference of a partial automaton. This is particularly useful when there is a need to constrain the time involved in the learning process

In [32]:
# Define a maximum time in seconds for the extraction
max_extraction_time = 20

# Create the General L* learner that implements the L* algorithm with the partial approach
learner = LStarFactory.get_partial_dfa_lstar_learner(max_time=max_extraction_time)

# Define an input alphabet
input_alphabet = Alphabet.from_strings(["a", "b"])

# Generate a random DFA with 1000 states
dfa = generate_dfa(input_alphabet, 1000, seed=100)

# Create the teacher for the DFA learner
dfa_teacher = GeneralTeacher(dfa, DFAComparisonStrategy())

# Learn the generated DFA
teacher = dfa_teacher
result = learner.learn(teacher) 

# Evaluate if the DFA was learned in the given time
print("Are equal: ", DFAComparisonStrategy().are_equivalent(
    result.model, dfa))

# Export the learned DFA
result.model.export('./output_models/dfa_partial_lstar/')

# Save the Observation Table
observation_table = result.info['observation_table']

Are equal:  False


#### Restart L*

We can use the restart approach to reuse the generated observation table from the last run of the algorithm.

In [33]:
# Learn the generated DFA with the saved Observation Table
result = learner.learn(teacher, observation_table=observation_table) 

# Evaluate if the DFA was learned in the given time
print("Are equal: ", DFAComparisonStrategy().are_equivalent(
    result.model, dfa))

Are equal:  True


## Observation Tree algorithms

### Observation Pack

#### Dependencies

In [34]:
from pymodelextractor.learners.observation_tree_learners.observation_pack_learner import ObservationPackLearner
from pymodelextractor.teachers.automaton_teacher import DeterministicFiniteAutomatonTeacher
from pythautomata.model_comparators.dfa_comparison_strategy import DFAComparisonStrategy
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars

#### Implementation

In [35]:
# Create the Observation Pack learner that implements the Observation Pack algorithm
learner = ObservationPackLearner()

# Import a sample DFA
dfa = TomitasGrammars.get_automaton_3()

# Create the teacher for the DFA learner
dfa_teacher = DeterministicFiniteAutomatonTeacher(dfa, DFAComparisonStrategy())

# Learn the generated DFA
teacher = dfa_teacher
result = learner.learn(teacher)

# Show that the learned DFA is equivalent to the generated DFA
print("Are equal: ", DFAComparisonStrategy().are_equivalent(
    result.model, dfa))

# Export the learned DFA
result.model.export('./output_models/dfa_observation_pack/')

Are equal:  True


### Kearns and Vazirani

#### Dependencies

In [36]:
from pymodelextractor.learners.observation_tree_learners.kearns_vazirani_learner import KearnsVaziraniLearner
from pymodelextractor.teachers.automaton_teacher import DeterministicFiniteAutomatonTeacher
from pythautomata.model_comparators.dfa_comparison_strategy import DFAComparisonStrategy
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars

#### Implementation

In [37]:
# Create the Observation Pack learner that implements the Kearns and Vazirani algorithm
learner = KearnsVaziraniLearner()

# Import a sample DFA
dfa = TomitasGrammars.get_automaton_4()

# Create the teacher for the DFA learner
dfa_teacher = DeterministicFiniteAutomatonTeacher(dfa, DFAComparisonStrategy())

# Learn the generated DFA
teacher = dfa_teacher
result = learner.learn(teacher)

# Show that the learned DFA is equivalent to the generated DFA
print("Are equal: ", DFAComparisonStrategy().are_equivalent(
    result.model, dfa))

# Export the learned DFA
result.model.export('./output_models/dfa_kearns_vazirani/')

Are equal:  True
